In [28]:
import pandas as pd
import seaborn as sns
import numpy as np
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

In [29]:
df_0 = pd.read_csv('dataset-s0.csv', delimiter=',')
df_0 = df_0.pivot_table('value', ['time'], 'variable')

In [30]:
df_1 = pd.read_csv('dataset-s1.csv', delimiter=',')
df_1 = df_1.pivot_table('value', ['time'], 'variable')

In [31]:
df_2 = pd.read_csv('dataset-o1.csv', delimiter=',')
df_2 = df_2.pivot_table('value', ['time'], 'variable')

In [32]:
df_3 = pd.read_csv('dataset-o2.csv', delimiter=',')
df_3 = df_3.pivot_table('value', ['time'], 'variable')

In [33]:
df_4 = pd.read_csv('dataset-s4.csv', delimiter=',')
df_4 = df_4.pivot_table('value', ['time'], 'variable')

In [34]:
df_5 = pd.read_csv('dataset-s5.csv', delimiter=',')
df_5 = df_5.pivot_table('value', ['time'], 'variable')

In [35]:
frames = [df_0, df_1, df_2, df_3,df_4]
df = pd.concat(frames)

In [36]:
df=df.dropna(axis=0)
df.isna().sum()

In [37]:
df.drop(columns=["servers", "utility"], axis = 1, inplace=True)

In [56]:
df.describe()

variable,activeServers,dimmer,requestRate,responseTime
count,1800.000000,1800.000000,1800.000000,1800.000000
mean,2.483889,0.977000,24.691132,0.791878
std,0.724974,0.070702,18.484947,2.437320
min,1.000000,0.300000,6.370498,0.011549
25%,2.000000,1.000000,8.022232,0.038394
50%,3.000000,1.000000,19.488683,0.041307
75%,3.000000,1.000000,35.136072,0.049256
max,3.000000,1.000000,84.228284,13.598175


## Training

In [38]:
features = ['activeServers', 'dimmer', 'requestRate']
X = df[features]
y = df['responseTime']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [40]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression().fit(X, y)

In [41]:
print(linear_reg.score(X_train, y_train))
print(linear_reg.score(X_test, y_test))

0.30123035917252294
0.30834252857412536


In [42]:
from sklearn.metrics import mean_squared_error
y_pred = linear_reg.predict(X_test)
print(mean_squared_error(y_test, y_pred))

4.342492174452417


In [43]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
rf_regr = RandomForestRegressor(max_depth=2, random_state=0)
rf_regr.fit(X, y)

RandomForestRegressor(max_depth=2, random_state=0)

In [44]:
print(rf_regr.score(X_train, y_train))
print(rf_regr.score(X_test, y_test))

0.7038759993245234
0.7922537103523244


In [45]:
y_pred = rf_regr.predict(X_test)
print(mean_squared_error(y_test, y_pred))

1.3043112730449826


In [46]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X, y)

KNeighborsRegressor(n_neighbors=2)

In [47]:
print(neigh.score(X_train, y_train))
print(neigh.score(X_test, y_test))

0.89190807088111
0.875559169140698


In [48]:
y_pred = neigh.predict(X_test)
print(mean_squared_error(y_test, y_pred))

0.7812874963598061


In [49]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [51]:
model = Sequential()
model.add(Dense(100, input_dim=3, activation= "relu", input_shape=[len(X_train.keys())]))
model.add(Dense(50, activation= "relu"))
model.add(Dense(0, activation= "relu"))
model.add(Dense(1))

In [52]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])


In [53]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
40/40 [==============================] - 1s 1ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 2/20
40/40 [==============================] - 0s 2ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 3/20
40/40 [==============================] - 0s 2ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 4/20
40/40 [==============================] - 0s 2ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 5/20
40/40 [==============================] - 0s 2ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 6/20
40/40 [==============================] - 0s 2ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 7/20
40/40 [==============================] - 0s 1ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 8/20
40/40 [==============================] - 0s 1ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 9/20
40/40 [==============================] - 0s 1ms/step - loss: 6.4071 - mean_squared_error: 6.4071
Epoch 10/20
40/40 [=========

In [54]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

40/40 [==============================] - 0s 1ms/step
2.4074816606581284
17/17 [==============================] - 0s 1ms/step
2.5074435176427223


In [55]:
model.save('keras_model.h5', include_optimizer=False)